# AI Diet Tracker - Model Training on Google Colab

Train a ResNet-50 food classifier on Food-101 dataset using **free GPU**.

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → **T4 GPU**
2. **Run all cells** (Runtime → Run all)
3. **Download trained model** at the end
4. Upload `food_classifier.pth` to your backend `models/` folder

**Training time:** ~30-40 minutes on free GPU

In [ ]:
# Install dependencies
!pip install torch torchvision tqdm datasets pillow -q

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from tqdm import tqdm
import time
import json
from datetime import datetime

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Configuration

In [ ]:
# Training config
EPOCHS = 10  # Reduce to 5 for faster training
BATCH_SIZE = 64  # GPU can handle larger batches
LEARNING_RATE = 0.001
NUM_CLASSES = 101  # Food-101 has 101 food categories

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Download Food-101 Dataset

In [ ]:
# Training transforms (data augmentation)
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Validation transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

print("Downloading Food-101 dataset... (this takes 5-10 minutes on first run)")
train_dataset = datasets.Food101(
    root='./data',
    split='train',
    transform=train_transform,
    download=True,
)

test_dataset = datasets.Food101(
    root='./data',
    split='test',
    transform=val_transform,
    download=True,
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"✅ Training samples: {len(train_dataset):,}")
print(f"✅ Test samples: {len(test_dataset):,}")

## Define Model Architecture

In [ ]:
class FoodClassifier(nn.Module):
    """ResNet-50 fine-tuned for food classification."""
    
    def __init__(self, num_classes=101):
        super().__init__()
        # Load pretrained ResNet-50
        self.backbone = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        
        # Replace final FC layer
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        return self.backbone(x)

model = FoodClassifier(num_classes=NUM_CLASSES)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

print(f"✅ Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

## Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress = tqdm(loader, desc='Training')
    for images, labels in progress:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        progress.set_postfix(
            loss=f"{loss.item():.4f}",
            acc=f"{100.0 * correct / total:.2f}%"
        )
    
    return running_loss / len(loader), 100.0 * correct / total


def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100.0 * correct / total

In [ ]:
# Training loop
best_accuracy = 0.0
training_history = []

print(f"\n🚀 Starting training for {EPOCHS} epochs...\n")

for epoch in range(EPOCHS):
    start_time = time.time()
    
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"{'='*60}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    
    # Step scheduler
    scheduler.step()
    
    epoch_time = time.time() - start_time
    
    print(f"\n📊 Epoch {epoch + 1} Results:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"  Time: {epoch_time:.1f}s")
    
    training_history.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'time_seconds': epoch_time,
    })
    
    # Save best model
    if val_acc > best_accuracy:
        best_accuracy = val_acc
        print(f"  ✅ New best model! (Val Acc: {val_acc:.2f}%)")

print(f"\n\n{'='*60}")
print(f"🎉 Training Complete!")
print(f"{'='*60}")
print(f"Best Validation Accuracy: {best_accuracy:.2f}%")

## Save Model

In [ ]:
# Food-101 class names
FOOD101_CLASSES = [
    "apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio", "beef_tartare",
    "beet_salad", "beignets", "bibimbap", "bread_pudding", "breakfast_burrito",
    "bruschetta", "caesar_salad", "cannoli", "caprese_salad", "carrot_cake",
    "ceviche", "cheese_plate", "cheesecake", "chicken_curry", "chicken_quesadilla",
    "chicken_wings", "chocolate_cake", "chocolate_mousse", "churros", "clam_chowder",
    "club_sandwich", "crab_cakes", "creme_brulee", "croque_madame", "cup_cakes",
    "deviled_eggs", "donuts", "dumplings", "edamame", "eggs_benedict",
    "escargots", "falafel", "filet_mignon", "fish_and_chips", "foie_gras",
    "french_fries", "french_onion_soup", "french_toast", "fried_calamari", "fried_rice",
    "frozen_yogurt", "garlic_bread", "gnocchi", "greek_salad", "grilled_cheese_sandwich",
    "grilled_salmon", "guacamole", "gyoza", "hamburger", "hot_and_sour_soup",
    "hot_dog", "huevos_rancheros", "hummus", "ice_cream", "lasagna",
    "lobster_bisque", "lobster_roll_sandwich", "macaroni_and_cheese", "macarons", "miso_soup",
    "mussels", "nachos", "omelette", "onion_rings", "oysters",
    "pad_thai", "paella", "pancakes", "panna_cotta", "peking_duck",
    "pho", "pizza", "pork_chop", "poutine", "prime_rib",
    "pulled_pork_sandwich", "ramen", "ravioli", "red_velvet_cake", "risotto",
    "samosa", "sashimi", "scallops", "seaweed_salad", "shrimp_and_grits",
    "spaghetti_bolognese", "spaghetti_carbonara", "spring_rolls", "steak", "strawberry_shortcake",
    "sushi", "tacos", "takoyaki", "tiramisu", "tuna_tartare", "waffles",
]

# Save checkpoint
checkpoint = {
    'model_state_dict': model.state_dict(),
    'num_classes': NUM_CLASSES,
    'class_names': FOOD101_CLASSES,
    'accuracy': best_accuracy,
    'epochs_trained': EPOCHS,
    'training_history': training_history,
    'trained_at': datetime.now().isoformat(),
}

torch.save(checkpoint, 'food_classifier.pth')
print(f"✅ Model saved to food_classifier.pth")

# Save training report
report = {
    'completed_at': datetime.now().isoformat(),
    'best_accuracy': best_accuracy,
    'total_epochs': EPOCHS,
    'history': training_history,
}

with open('training_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print(f"✅ Training report saved to training_report.json")

## Download Trained Model

In [ ]:
from google.colab import files

print("📥 Downloading trained model...")
files.download('food_classifier.pth')
files.download('training_report.json')

print("\n✅ Downloaded! Upload food_classifier.pth to:")
print("   /Users/amsan/app2026/backend/models/food_classifier.pth")
print("\nThen in .env, set:")
print("   USE_CUSTOM_MODEL=true")